In [ ]:
from partitioning_helper import *
import pandas
import time
from img_comm_get import *


pandas.set_option('display.max_rows', None)


while(True):
    # Load and parse json data produced by Darknet
    all_images = load_json('mdpfiles\\predicted_output\\result.json')
    # Create initial df using json data
    df = declare_df(all_images)
    # Determine distance offsets and update df with new column
    df = calculate_distance_offset(df)
    # Get paths to each image in input directory
    image_list = get_filelist('mdpfiles\\samples')
    # Create image_df using image_list and get initial dimensions for each image in directory
    image_df = declare_image_df(image_list)
    # Create concatenated df using df and image_df
    final_df = pd.concat([df, image_df], axis=1, sort=False)
    # Calculate Left and Right Boundaries
    final_df = calculate_partitions(final_df)
    # Determine offset using L/R Boundaries
    final_df = determine_offsets(final_df)
    # Store received robot x,y,o values from buffer into final_df
    final_df = retrieve_buffer_values(final_df)
    # Calculate actual x and y axis values
    final_df = calculate_actual_coordinates(final_df)
    # Map actual image ID
    final_df = map_actual_image_id(final_df)
    # Create df using only unique first instances
    unique_df = declare_unique_df(final_df)
    # Create output_df as a subset of the final_df, then iterate through and create output string
    output_string = declare_output_string(unique_df)


    '''
    Communication function
    '''
    print("main")
    rec = RecAPI()
    rec.init_pc_comm()

    '''
    rec.read_from_RPI("test.jpg")
    print("Image is saved")
    '''
    send_msg = output_string # uses output string from previous functions
    print("write_to_Android(): ", send_msg)
    rec.write_to_Android(send_msg.encode())

    print("closing sockets")
    rec.close_pc_socket()
    
    
    time.sleep(1)

In [ ]:
output_string

In [15]:
import json
import pandas as pd
import numpy as np
import time
import os

def load_json(json_directory):
    with open(json_directory) as json_file:
        data = json.load(json_file)
        all_images = []
        for each in data:
            pathname = each['filename']
            if (len(each['objects']) != 0):
                print(each['objects'])
                name = each['objects'][0]['name']
                x_axis = each['objects'][0]['relative_coordinates']['left_x']
                y_axis = each['objects'][0]['relative_coordinates']['top_y']
                box_width = each['objects'][0]['relative_coordinates']['width']
                box_height = each['objects'][0]['relative_coordinates']['height']
                box_area = box_width * box_height
                confidence = each['objects'][0]['confidence']
                image_info = [pathname, name, x_axis, y_axis, box_width, box_height, box_area, confidence]
                all_images.append(image_info)
load_json('mdpfiles/predicted_output/result.json')